# 🎸 Guitar Strum Generator - Dataset Construction

**Notebook 02: Build Training Dataset**

This notebook creates the training dataset for your thesis by combining:
1. **Synthetic samples (70%)** - Generated using the rule-based system
2. **Real progressions (30%)** - Sampled from Chordonomicon dataset

---

**Author:** Rohan Rajendra Dhanawade  
**Thesis:** A Conversational AI System for Symbolic Guitar Strumming Pattern and Chord Progression Generation

---

## 1. Setup Environment

First, let's clone your repository and install dependencies.

In [3]:
# Clone repository (if running in Colab)
import os

if not os.path.exists('guitar-strum-gen'):
    !git clone https://github.com/rohand575/guitar-strum-gen.git
    %cd guitar-strum-gen
else:
    %cd guitar-strum-gen
    !git pull

print("\n✅ Repository ready!")

Cloning into 'guitar-strum-gen'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 78 (delta 16), reused 71 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 51.86 KiB | 6.48 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/guitar-strum-gen/guitar-strum-gen/guitar-strum-gen

✅ Repository ready!


In [4]:
# Install required packages
!pip install -q pydantic datasets pandas tqdm

# Install the project in development mode
!pip install -q -e .

print("\n✅ Dependencies installed!")

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for guitar-strum-gen (pyproject.toml) ... done

✅ Dependencies installed!


In [5]:
# Verify imports work
from src.data.schema import GuitarSample, VALID_GENRES, VALID_EMOTIONS
from src.data.build_dataset import (
    DatasetConfig, 
    build_dataset,
    generate_synthetic_sample,
    load_chordonomicon_huggingface,
    PROMPT_TEMPLATES
)

print(f"✅ All imports successful!")
print(f"   - {len(VALID_GENRES)} genres available")
print(f"   - {len(VALID_EMOTIONS)} emotions available")
print(f"   - {len(PROMPT_TEMPLATES)} prompt templates")

✅ All imports successful!
   - 9 genres available
   - 8 emotions available
   - 68 prompt templates


## 2. Explore Chordonomicon Dataset

Let's load and explore the Chordonomicon dataset from Hugging Face.

In [6]:
# Load Chordonomicon from Hugging Face
from datasets import load_dataset

print("📥 Loading Chordonomicon dataset from Hugging Face...")
print("   (This may take a few minutes on first run)\n")

chordonomicon = load_dataset("ailsntua/Chordonomicon", split="train")

print(f"✅ Loaded {len(chordonomicon):,} chord progressions!")
print(f"\nColumns: {chordonomicon.column_names}")

📥 Loading Chordonomicon dataset from Hugging Face...
   (This may take a few minutes on first run)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


✅ Loaded 679,807 chord progressions!

Columns: ['id', 'chords', 'release_date', 'genres', 'decade', 'rock_genre', 'artist_id', 'main_genre', 'spotify_song_id', 'spotify_artist_id']


In [7]:
# Convert to pandas for easier exploration
import pandas as pd

df = chordonomicon.to_pandas()

print("📊 Dataset Overview:")
print(f"   Total entries: {len(df):,}")
print(f"   Memory usage: {df.memory_usage().sum() / 1024**2:.1f} MB")
print("\n📋 First few entries:")
df.head(3)

📊 Dataset Overview:
   Total entries: 679,807
   Memory usage: 51.9 MB

📋 First few entries:


,id,chords,release_date,genres,decade,rock_genre,artist_id,main_genre,spotify_song_id,spotify_artist_id
0,1,<intro_1> C <verse_1> F C E7 Amin C F C G7 C F...,None,'classic country pop',NaN,None,artist_1,pop,None,4AIEGdwDzPELXYgM5JaEY5
1,2,<intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs...,2003-01-01,'alternative metal' 'alternative rock' 'nu met...,2000.0,pop rock,artist_2,metal,2ffJZ2r8HxI5DHcmf3BO6c,694QW15WkebjcrWgQHzRYF
2,3,<intro_1> Csmin <verse_1> A Csmin A Csmin A Cs...,2003-01-01,'alternative metal' 'canadian rock' 'funk meta...,2000.0,canadian rock,artist_3,metal,5KiY8SZEnvCPyIEkFGRR3y,0niJkG4tKkne3zwr7I8n9n


In [8]:
# Let's look at a few example chord progressions
print("🎵 Example chord progressions from Chordonomicon:\n")

for i in range(5):
    row = df.iloc[i]
    chords = row['chords'][:200] + "..." if len(str(row['chords'])) > 200 else row['chords']
    print(f"Example {i+1}:")
    print(f"  Genre: {row.get('main_genre', 'N/A')}")
    print(f"  Chords: {chords}")
    print()

🎵 Example chord progressions from Chordonomicon:

Example 1:
  Genre: pop
  Chords: <intro_1> C <verse_1> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <verse_2> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <chorus_1> F C F C G C F C E7 Amin C F G7 C <solo_1> D <chorus_2> G D G D A D G ...

Example 2:
  Genre: metal
  Chords: <intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_1> E G D A E G D A E G D A C D E D A/Cs <verse_2> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_2> E G D A E G D A E G D A...

Example 3:
  Genre: metal
  Chords: <intro_1> Csmin <verse_1> A Csmin A Csmin A Csmin A B <chorus_1> Csmin A Fsmin A B <verse_2> Csmin A Csmin A Csmin A Csmin A Fsmin <chorus_2> Csmin A Fsmin A B Csmin A Fsmin Csmin <bridge_1> A Csmin A...

Example 4:
  Genre: None
  Chords: <intro_1> D Dmaj7 D Dmaj7 <verse_1> Emin A D G Emin A D G Emin A D G Emin A D <chorus_1> Emin A D G Emin A D G D/Fs Emin A D G D/Fs Emin A D G <verse_2> Emin A D G Emin A D G Emin A D G Emin A D

In [9]:
# Genre distribution in Chordonomicon
if 'main_genre' in df.columns:
    print("📊 Genre distribution in Chordonomicon:")
    genre_counts = df['main_genre'].value_counts().head(15)
    for genre, count in genre_counts.items():
        pct = count / len(df) * 100
        print(f"   {genre:20} {count:>8,} ({pct:5.1f}%)")

📊 Genre distribution in Chordonomicon:
   pop                    85,185 ( 12.5%)
   rock                   67,238 (  9.9%)
   country                53,306 (  7.8%)
   alternative            47,252 (  7.0%)
   pop rock               39,557 (  5.8%)
   punk                   16,066 (  2.4%)
   metal                  11,315 (  1.7%)
   rap                    11,186 (  1.6%)
   soul                    7,350 (  1.1%)
   jazz                    7,001 (  1.0%)
   reggae                  3,841 (  0.6%)
   electronic              2,814 (  0.4%)


## 3. Test Chord Parsing

Let's test our chord parsing functions on real Chordonomicon data.

In [10]:
from src.data.build_dataset import parse_chordonomicon_chord_string, normalize_chord

# Test parsing on a few examples
print("🔧 Testing chord parsing on Chordonomicon samples:\n")

for i in range(5):
    row = df.iloc[i * 1000]  # Sample every 1000th row for variety
    original = str(row['chords'])[:100]
    chords, section = parse_chordonomicon_chord_string(str(row['chords']))
    
    print(f"Example {i+1}:")
    print(f"  Original: {original}...")
    print(f"  Parsed ({len(chords)} chords): {chords[:8]}")
    print(f"  Section: {section}")
    print()

🔧 Testing chord parsing on Chordonomicon samples:

Example 1:
  Original: <intro_1> C <verse_1> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <verse_2> F C E7 Amin C F C G7 C F...
  Parsed (66 chords): ['C', 'F', 'C', 'E7', 'Am', 'C', 'F', 'C']
  Section: intro_1

Example 2:
  Original: <verse_1> A D A D A <chorus_1> Dmin <bridge_1> Emin G <verse_2> A D A D A Dmin <bridge_2> Emin G A D...
  Parsed (20 chords): ['A', 'D', 'A', 'D', 'A', 'Dm', 'Em', 'G']
  Section: verse_1

Example 3:
  Original: <intro_1> Emin D Amin7 Emin <verse_1> D C Emin D C G D C Emin G D C Emin <interlude_1> D C Emin <ver...
  Parsed (59 chords): ['Em', 'D', 'Am7', 'Em', 'D', 'C', 'Em', 'D']
  Section: intro_1

Example 4:
  Original: <intro_1> Bmin Emin Fsmin <verse_1> Bmin Emin Fsmin Bmin Emin Fsmin <verse_2> Bmin Emin Fsmin Bmin E...
  Parsed (46 chords): ['Bm', 'Em', 'Bm', 'Em', 'Bm', 'Em', 'Bm', 'Em']
  Section: intro_1

Example 5:
  Original: <verse_1> G C F C Amin G C F C Amin C Amin G Amin G C F C Amin <verse

## 4. Generate Training Dataset

Now let's build the complete training dataset!

In [11]:
# Configuration for dataset generation
config = DatasetConfig(
    total_samples=250,        # Total samples to generate
    synthetic_ratio=0.70,     # 70% synthetic, 30% from Chordonomicon
    train_ratio=0.70,         # 70% train, 15% val, 15% test
    val_ratio=0.15,
    test_ratio=0.15,
    random_seed=42
)

print("📊 Dataset Configuration:")
print(f"   Total samples: {config.total_samples}")
print(f"   Synthetic: {config.num_synthetic} ({config.synthetic_ratio*100:.0f}%)")
print(f"   From Chordonomicon: {config.num_real} ({(1-config.synthetic_ratio)*100:.0f}%)")
print(f"   Train/Val/Test: {config.train_ratio*100:.0f}%/{config.val_ratio*100:.0f}%/{config.test_ratio*100:.0f}%")

📊 Dataset Configuration:
   Total samples: 250
   Synthetic: 175 (70%)
   From Chordonomicon: 75 (30%)
   Train/Val/Test: 70%/15%/15%


In [12]:
# Build the dataset!
import random
random.seed(config.random_seed)

splits = build_dataset(
    config=config,
    chordonomicon_path=None,  # We'll use Hugging Face
    use_huggingface=True,
    output_dir="data/processed"
)

🎸 GUITAR STRUM GENERATOR - DATASET BUILDER

Configuration:
  Total samples: 250
  Synthetic: 175 (70%)
  Real (Chordonomicon): 75 (30%)

--- Generating 175 synthetic samples ---
  ✅ Generated 175 synthetic samples

--- Loading 75 real progressions from Hugging Face ---
  📥 Loading from Hugging Face (ailsntua/Chordonomicon)...
  ✓ Loaded 679807 total entries from Chordonomicon
  ✓ 679668 entries have valid chord data
  ✓ Successfully parsed 10 progressions
  ✅ Generated 10 samples from Chordonomicon

--- Dataset Splits ---
  Train: 129 samples
  Val:   27 samples
  Test:  29 samples

  ✅ Saved: data/processed/dataset.jsonl
  ✅ Saved: data/processed/train.jsonl
  ✅ Saved: data/processed/val.jsonl
  ✅ Saved: data/processed/test.jsonl
  ✅ Saved: data/processed/stats.json

✅ Dataset generation complete!


In [11]:
# View generated files
!ls -la data/processed/

## 5. Explore Generated Dataset

In [ ]:
# Load and explore the generated dataset
import json

# Load stats
with open('data/processed/stats.json', 'r') as f:
    stats = json.load(f)

print("📊 Generated Dataset Statistics:")
print(f"\n   Total samples: {stats['total']}")
print(f"   Unique progressions: {stats['unique_progressions']}")
print(f"   Unique patterns: {stats['unique_patterns']}")
print(f"   Tempo range: {stats['tempo_range'][0]} - {stats['tempo_range'][1]} BPM")
print(f"   Average tempo: {stats['avg_tempo']:.1f} BPM")

print(f"\n📁 Splits:")
for split, count in stats['splits'].items():
    print(f"   {split}: {count} samples")

In [ ]:
# Genre distribution
print("🎵 Genre Distribution:")
for genre, count in sorted(stats['genres'].items(), key=lambda x: x[1], reverse=True):
    pct = count / stats['total'] * 100
    bar = '█' * int(pct / 2)
    print(f"   {genre:12} {count:3} ({pct:5.1f}%) {bar}")

In [ ]:
# Emotion distribution
print("💭 Emotion Distribution:")
for emotion, count in sorted(stats['emotions'].items(), key=lambda x: x[1], reverse=True):
    pct = count / stats['total'] * 100
    bar = '█' * int(pct / 2)
    print(f"   {emotion:12} {count:3} ({pct:5.1f}%) {bar}")

In [ ]:
# View some sample entries
print("📝 Sample Entries from Dataset:\n")

with open('data/processed/dataset.jsonl', 'r') as f:
    samples = [json.loads(line) for line in f.readlines()[:10]]

for i, sample in enumerate(samples[:5]):
    print(f"--- Sample {i+1} ({sample['id']}) ---")
    print(f"Prompt: \"{sample['prompt']}\"")
    print(f"Chords: {sample['chords']}")
    print(f"Pattern: {sample['strum_pattern']}")
    print(f"Key: {sample['key']} {sample['mode']} | Tempo: {sample['tempo']} BPM")
    print(f"Genre: {sample['genre']} | Emotion: {sample['emotion']}")
    print()

## 6. Visualize Dataset Distribution

In [ ]:
import matplotlib.pyplot as plt

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Genre distribution
ax1 = axes[0, 0]
genres = list(stats['genres'].keys())
genre_counts = list(stats['genres'].values())
ax1.barh(genres, genre_counts, color='steelblue')
ax1.set_xlabel('Count')
ax1.set_title('Genre Distribution')

# Emotion distribution
ax2 = axes[0, 1]
emotions = list(stats['emotions'].keys())
emotion_counts = list(stats['emotions'].values())
ax2.barh(emotions, emotion_counts, color='coral')
ax2.set_xlabel('Count')
ax2.set_title('Emotion Distribution')

# Mode distribution (pie)
ax3 = axes[1, 0]
modes = list(stats['modes'].keys())
mode_counts = list(stats['modes'].values())
ax3.pie(mode_counts, labels=modes, autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
ax3.set_title('Mode Distribution (Major vs Minor)')

# Key distribution
ax4 = axes[1, 1]
keys = list(stats['keys'].keys())
key_counts = list(stats['keys'].values())
ax4.bar(keys, key_counts, color='purple', alpha=0.7)
ax4.set_xlabel('Key')
ax4.set_ylabel('Count')
ax4.set_title('Key Distribution')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('data/processed/dataset_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Visualization saved to data/processed/dataset_distribution.png")

## 7. Download Dataset Files

Download the generated files to your local machine.

In [ ]:
# Create a zip file for easy download
!cd data/processed && zip -r ../../guitar_dataset.zip *.jsonl *.json *.png 2>/dev/null

print("📦 Created guitar_dataset.zip")
print("\nContents:")
!unzip -l guitar_dataset.zip

In [ ]:
# Download (only works in Colab)
try:
    from google.colab import files
    files.download('guitar_dataset.zip')
    print("\n✅ Download started!")
except ImportError:
    print("\n📁 Not running in Colab. Files are in data/processed/")
    print("   You can manually download:")
    print("   - train.jsonl")
    print("   - val.jsonl")
    print("   - test.jsonl")
    print("   - stats.json")

## 8. Summary

### What We Built

✅ **Training dataset** with 250 samples:
- 175 synthetic samples (70%) from rule-based system
- 75 real progressions (30%) from Chordonomicon

✅ **Rich prompt diversity** with 68 unique templates

✅ **Balanced distribution** across:
- 9 genres (pop, rock, folk, ballad, country, blues, jazz, indie, acoustic)
- 8 emotions (upbeat, melancholic, mellow, energetic, peaceful, dramatic, hopeful, nostalgic)
- Major and minor keys

### Output Files

| File | Description |
|------|-------------|
| `train.jsonl` | Training set (70%) |
| `val.jsonl` | Validation set (15%) |
| `test.jsonl` | Test set (15%) |
| `dataset.jsonl` | Complete dataset |
| `stats.json` | Dataset statistics |

### Next Steps

Continue to **Notebook 03: Training** to train the neural sequence model!

In [ ]:
print("🎉 Dataset construction complete!")
print("\n📊 Final Stats:")
print(f"   - {stats['total']} total samples")
print(f"   - {stats['unique_progressions']} unique chord progressions")
print(f"   - {stats['unique_patterns']} unique strumming patterns")
print(f"   - Train/Val/Test: {stats['splits']['train']}/{stats['splits']['val']}/{stats['splits']['test']}")
print("\n🚀 Ready for neural model training!")